In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 폰트 설정이 잘 안 된다면 해당 셀의 마지막 줄 주석을 풀고 실행해 보세요.
# colab 사용시에도 아래 함수를 활용해 보세요.
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())

In [ ]:
# Win
# plt.rc("font", family="Malgun Gothic")
# Mac
# plt.rc("font", family="AppleGothic")
plt.rc("axes", unicode_minus=False)

# 폰트가 선명하게 보이도록 설정

from IPython.display import set_matplotlib_formats

set_matplotlib_formats("retina")

1. 데이터 로드 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 소상공인 상가 정보
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Cp1_1/소상공인시장진흥공단_상가(상권)정보_서울_202203.csv")
df.head()

In [ ]:
columns = ['상호명', '상권업종대분류명', '상권업종중분류명', '상권업종소분류명', 
           '시도명', '시군구명', '행정동명', '법정동명', '도로명주소', 
           '경도', '위도']
print(df.shape)
df = df[columns].copy()
df.shape

In [ ]:
df_seoul = df[df["시도명"] == "서울특별시"].copy()
print(df_seoul.shape)
df_seoul.head()

In [ ]:
df_seoul["행정동명"].unique()

In [ ]:
df_seoul["행정동명"].nunique()

In [ ]:
df_seoul["상호명_소문자"] = df_seoul["상호명"].str.lower()

In [ ]:
df_seoul.loc[df_seoul["상호명_소문자"].str.contains("파파이스"), "상호명_소문자"].shape

In [ ]:
df_seoul.loc[df_seoul["상호명_소문자"].str.contains("프랭크버거"), "상호명_소문자"].shape

In [ ]:
df_seoul.loc[df_seoul["상호명_소문자"].str.contains("쉐이크쉑"), "상호명_소문자"].shape

In [ ]:
df_seoul["상호명_소문자"] = df_seoul["상호명"].str.lower()

df_ham = df_seoul[
    df_seoul["상호명_소문자"].str.contains('맥도날드|버거킹|맘스터치|롯데리아|kfc|노브랜드버거|쉐이크쉑|프랭크버거|파파이스')].copy()
df_ham.shape





In [ ]:
df_ham.loc[df_ham["상호명_소문자"].str.contains("맥도날드"), 
          "브랜드명"] = "맥도날드"
df_ham.loc[df_ham["상호명_소문자"].str.contains("롯데리아"), 
          "브랜드명"] = "롯데리아"
df_ham.loc[df_ham["상호명_소문자"].str.contains("버거킹"), 
          "브랜드명"] = "버거킹"  
df_ham.loc[df_ham["상호명_소문자"].str.contains("맘스터치"), 
          "브랜드명"] = "맘스터치"        
df_ham.loc[df_ham["상호명_소문자"].str.contains("kfc"), 
          "브랜드명"] = "kfc"
df_ham.loc[df_ham["상호명_소문자"].str.contains("프랭크버거"), 
          "브랜드명"] = "프랭크버거"
df_ham.loc[df_ham["상호명_소문자"].str.contains("노브랜드버거"), 
          "브랜드명"] = "노브랜드버거"
df_ham.loc[df_ham["상호명_소문자"].str.contains("쉐이크쉑"), 
          "브랜드명"] = "쉐이크쉑"
df_ham.loc[df_ham["상호명_소문자"].str.contains("파파이스"), 
          "브랜드명"] = "파파이스"

df_ham["브랜드명"].unique()

In [ ]:
#도로명 주소, 브랜드 중복 제거
df_ham = df_ham.drop_duplicates(['도로명주소','브랜드명'])
df_ham.shape


In [ ]:
df_ham = df_ham[['행정동명','브랜드명','시군구명','경도','위도','도로명주소','상호명']]

In [ ]:
plt.figure(figsize=(15, 4))


g = sns.countplot(data=df_ham, x="브랜드명", order = df_ham['브랜드명'].value_counts().index)

for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width() / 2., height + 3, height, ha = 'center', size = 9)

g.set_ylim(-5, 100)
plt.show()

In [ ]:
df_ham_rank= df_ham.pivot_table(index="브랜드명", values="도로명주소", aggfunc="count")
df_ham_rank.sort_values(by='도로명주소' ,ascending=False)

In [ ]:
plt.figure(figsize=(15, 4))
g = sns.countplot(data=df_ham, x="시군구명", hue="브랜드명")

In [ ]:
dong_ham_brand_count = df_ham.pivot_table(index="행정동명", values="브랜드명", aggfunc="count")
dong_ham_brand_count  = dong_ham_brand_count.sort_values(by='브랜드명' ,ascending=False)
dong_ham_brand_count.head(20)

In [ ]:
dong_ham_brand_count.reset_index(drop = False, inplace = True)

dong_ham_brand_count= dong_ham_brand_count[['행정동명','브랜드명']]
dong_ham_brand_count.columns=['name','ham_count']
dong_ham_brand_count

In [ ]:
dong_ham_brand = df_ham.pivot_table(index="행정동명", columns="브랜드명", values="경도", aggfunc="count")
dong_ham_brand

In [ ]:
sns.scatterplot(data=df_ham, x="경도", y="위도", hue="브랜드명")

In [ ]:
sns.jointplot(data=df_ham, x="경도", y="위도", kind="hex")

1.2 행정동별인구


In [ ]:
pop_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Cp1_1/동별인구.csv")

pop_df.head()

In [ ]:
pop_df = pop_df.astype({'인구합계':'int','남자':'int','여자':'int','65세이상':'int'})
pop_df.dtypes

In [ ]:
pop_dong_df = pop_df[['동', '인구합계','성비 ','65세이하 인구비율']]

pop_dong_df.columns=['name','values', 'sex_rate','young_rate']
pop_dong_df

In [ ]:
all_df=pd.merge(left = pop_dong_df , right = dong_ham_brand_count, how = "left", on = "name")
all_df = all_df.fillna(0)
all_df

In [ ]:
all_df = all_df.astype({'ham_count':'int'})
all_df.sort_values(by='ham_count' ,ascending=False)

In [ ]:
all_df.loc[all_df.name=='종로1.2.3.4가동','name'] = '종로1·2·3·4가동'
all_df.loc[all_df.name=='금호2.3가동','name'] = '금호2·3가동'
all_df.loc[all_df.name=='중계2.3동','name'] = '중계2·3동'
all_df.loc[all_df.name=='상계3.4동','name'] = '상계3·4동'
all_df.loc[all_df.name=='상계6.7동','name'] = '상계6·7동'
all_df.loc[all_df.name=='면목3.8동','name']='면목3·8동'

#상일동 = 상일1동 2동 합친다
all_df.loc[all_df.name=='상일동','ham_count'] = 2


all_df.sort_values(by='ham_count' ,ascending=False)

1.3 행정동별 5월 생활인구 평균(시간,날짜)

In [ ]:
life_pop_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Cp1_1/LOCAL_PEOPLE_DONG_202205_.csv")
life_pop_df

In [ ]:
dong_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Cp1_1/행정동코드_매핑정보_2018_.csv")
dong_df

In [ ]:
life_pop_dong_df=pd.merge(left = life_pop_df , right = dong_df, how = "left", on = "행정동코드")
life_pop_dong_df = life_pop_dong_df[['행정동','생활인구_avg']]
life_pop_dong_df.columns=['name','local_avg']
life_pop_dong_df

In [ ]:
life_pop_dong_df.loc[life_pop_dong_df.name=='종로1.2.3.4가동','name'] = '종로1·2·3·4가동'
life_pop_dong_df.loc[life_pop_dong_df.name=='종로5.6가동','name'] = '종로5·6가동'
life_pop_dong_df.loc[life_pop_dong_df.name=='금호2.3가동','name'] = '금호2·3가동'
life_pop_dong_df.loc[life_pop_dong_df.name=='중계2.3동','name'] = '중계2·3동'
life_pop_dong_df.loc[life_pop_dong_df.name=='상계3.4동','name'] = '상계3·4동'
life_pop_dong_df.loc[life_pop_dong_df.name=='상계6.7동','name'] = '상계6·7동'
life_pop_dong_df.loc[life_pop_dong_df.name=='면목3.8동','name']='면목3·8동'

In [ ]:
all_final_df=pd.merge(left = all_df , right = life_pop_dong_df, how = "left", on = "name")
all_final_df= all_final_df.drop_duplicates(['name'])

all_final_df =all_final_df.dropna(axis=0)
all_final_df

2. 시각화

In [ ]:
import folium
import json

geo_json_dong ='https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_submunicipalities_geo_simple.json'  


In [ ]:
# 지도의 중심을 지정하기 위해 위도와 경도의 평균을 구합니다. 
lat = df_ham["위도"].mean()
long = df_ham["경도"].mean()
lat, long

In [ ]:
from folium.plugins import MarkerCluster

mm = folium.Map([lat, long], zoom_start=11)
#marker_cluster = MarkerCluster().add_to(mm)


folium.GeoJson(
    geo_json_dong,
    name='seoul_municipalities'
).add_to(mm)


for i in df_ham.index:
    
    sub_lat = df_ham.loc[i, "위도"]
    sub_long = df_ham.loc[i, "경도"]
    title = df_ham.loc[i, "상호명"] + " - " + df_ham.loc[i, "도로명주소"]
    
    icon_color = "blue"
    if df_ham.loc[i, "브랜드명"] == "맥도날드":
        icon_color = "red"

    if df_ham.loc[i, "브랜드명"] == "버거킹":
        icon_color = "brown"

    if df_ham.loc[i, "브랜드명"] == "kfc":
        icon_color = "green"

    if df_ham.loc[i, "브랜드명"] == "맘스터치":
        icon_color = "purple"

    if df_ham.loc[i, "브랜드명"] == "노브랜드버거":
        icon_color = "yellow"


    if df_ham.loc[i, "브랜드명"] == "쉐이크쉑":
        icon_color = "lightgreen"


    if df_ham.loc[i, "브랜드명"] == "파파이스":
        icon_color = "lightgray"

    if df_ham.loc[i, "브랜드명"] == "프랭크버거":
        icon_color = "darkpurple"



    folium.Marker(
        [sub_lat, sub_long], 
        icon=folium.Icon(color=icon_color),
        popup=f'<i>{title}</i>', 
        tooltip=title).add_to(mm)

mm.save('index.html')
mm

In [ ]:
from folium.plugins import MarkerCluster

mc = folium.Map([lat, long], zoom_start=11)
marker_cluster = MarkerCluster().add_to(mc)


folium.GeoJson(
    geo_json_dong,
    name='seoul_municipalities'
).add_to(mc)


for i in df_ham.index:
    
    sub_lat = df_ham.loc[i, "위도"]
    sub_long = df_ham.loc[i, "경도"]
    title = df_ham.loc[i, "상호명"] + " - " + df_ham.loc[i, "도로명주소"]
    
    icon_color = "blue"
    if df_ham.loc[i, "브랜드명"] == "맥도날드":
        icon_color = "red"

    if df_ham.loc[i, "브랜드명"] == "버거킹":
        icon_color = "brown"

    if df_ham.loc[i, "브랜드명"] == "kfc":
        icon_color = "green"

    if df_ham.loc[i, "브랜드명"] == "맘스터치":
        icon_color = "purple"

    if df_ham.loc[i, "브랜드명"] == "노브랜드버거":
        icon_color = "yellow"


    if df_ham.loc[i, "브랜드명"] == "쉐이크쉑":
        icon_color = "lightgreen"


    if df_ham.loc[i, "브랜드명"] == "파파이스":
        icon_color = "lightgray"

    if df_ham.loc[i, "브랜드명"] == "프랭크버거":
        icon_color = "darkpurple"



    folium.Marker(
        [sub_lat, sub_long], 
        icon=folium.Icon(color=icon_color),
        popup=f'<i>{title}</i>', 
        tooltip=title).add_to(marker_cluster)

mc.save('index.html')
mc

In [ ]:
# 주민등록 기반 인구수 
n=folium.Map(
    [lat, long],
    zoom_start=11,
    tiles='cartodbpositron'
)

folium.Choropleth(
    geo_data=geo_json_dong,
    name='choropleth',
    data=all_final_df,
    columns=['name','values'],
    key_on='feature.properties.name',
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.3,


).add_to(n)

n

In [ ]:
all_final_df.sort_values(by='values' ,ascending=False)[['name','values']].head(10)

In [ ]:
# 행정동별 햄버거 전문점
ham=folium.Map(
    [lat, long],
    zoom_start=11,
    tiles='cartodbpositron'
)

folium.Choropleth(
    geo_data=geo_json_dong,
    name='choropleth',
    data=all_final_df,
    columns=['name','ham_count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.3,


).add_to(ham)

ham

In [ ]:
all_final_df.sort_values(by='ham_count' ,ascending=False)[['name','ham_count']].head(10)

In [ ]:
# 행정동별 생활인구 평균
local=folium.Map(
    [lat, long],
    zoom_start=12,
    tiles='cartodbpositron'
)

folium.Choropleth(
    geo_data=geo_json_dong,
    name='choropleth',
    data=all_final_df,
    columns=['name','local_avg'],
    key_on='feature.properties.name',
    fill_color='YlGn', 
    fill_opacity=0.7, 
    line_opacity=0.3,


).add_to(local)

local

In [ ]:
all_final_df.sort_values(by='local_avg' ,ascending=False)[['name','local_avg']].head(10)

##3. 데이터 분석


In [ ]:
all_final_df

In [ ]:
multi_reg_df= all_final_df.drop(['name'], axis=1)
multi_reg_df

In [ ]:
multi_reg_df.dtypes

In [ ]:
multi_reg_df.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(multi_reg_df, test_size=0.2)

label = 'ham_count'
feature = ['values',	'sex_rate',	'young_rate','local_avg']

X_train = train[feature]
y_train = train[label]

X_test = test[feature]
y_test = test[label]


In [ ]:
X_test

In [ ]:
multi_reg_df['local_avg'].mean() 

In [ ]:
multi_reg_df['values'].mean() 

In [ ]:
all_final_df.loc[all_final_df['name']=='가락본동']

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

mean_base =multi_reg_df['ham_count'].mean() # 평균 base line

mean_base_to_insert = {'Price': mean_base}
mean_base_df = pd.DataFrame({'Price': [mean_base]})

for i in range(1, multi_reg_df['ham_count'].shape[0]):  # 계산을 위한 데이터프레임 생성
        mean_base_df = mean_base_df.append(mean_base_to_insert, ignore_index=True)
        
mean_base_df

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ['values',	'sex_rate',	'young_rate','local_avg','ham_count']
scatter_matrix(all_final_df[attributes], figsize=(12, 8))

In [ ]:
!pip install scipy

In [ ]:
import statsmodels.api as sm

lin_reg = sm.OLS(y_train, X_train).fit()
lin_reg.summary()

In [ ]:
!pip install --upgrade category_encoders

In [ ]:
from scipy.stats import randint, uniform
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

pre_process = make_pipeline(
        OrdinalEncoder(), 
        SimpleImputer(strategy='median'),
        XGBClassifier()
)

# fit_transform, transform
#X_train_processed = pre_process.fit_transform(X_train)
#X_test_processed = pre_process.transform(X_test)

xgb = XGBClassifier()

dists = {
    'xgbclassifier__n_estimators': [100, 2000], 
    'xgbclassifier__max_depth': [1, 10], 
    'xgbclassifier__learning_rate' : [0.01, 0.05, 0.2, 0.3, 0.7, 1],
    'xgbclassifier__max_delta_step' : [1, 5],
    'xgbclassifier__min_child_weight': [1, 6.], 
    'xgbclassifier__gamma': [0, 0.5], 
    'xgbclassifier__subsample' : [0.5, 1.],
    'xgbclassifier__colsample_bytree' : [0.5, 1.]
}

random_clf = RandomizedSearchCV(
    pre_process,
    param_distributions=dists, 
    scoring='accuracy',
    verbose=1,
    n_iter=50, 
    cv=3, 
    n_jobs=-1,
)


random_clf.fit(X_train, y_train);


print('최적 하이퍼파라미터: ', random_clf.best_params_)
print('accuracy: ', random_clf.best_score_)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score
xgb_pipe = random_clf.best_estimator_
y_pred = xgb_pipe.predict(X_test)

print("\n <classification_report> \n ", classification_report(y_pred, y_test))

print('훈련세트 정확도 : ', xgb_pipe.score(X_train, y_train))
print('검증세트 정확도 : ', xgb_pipe.score(X_test, y_test))

In [ ]:

pipe = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=2)
)
dists = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1)
}
clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='accuracy',  
    verbose=1,
    n_jobs=-1
)
clf.fit(X_train, y_train);
print('최적 하이퍼파라미터: ', clf.best_params_)
print('accuracy: ', clf.best_score_)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score
ran_forest_pipe = clf.best_estimator_
y_pred = ran_forest_pipe.predict(X_test)


print("\n <classification_report> \n ", classification_report(y_pred, y_test))

print('훈련세트 정확도 : ', ran_forest_pipe.score(X_train, y_train))
print('검증세트 정확도 : ', ran_forest_pipe.score(X_test, y_test))

In [ ]:
!pip install eli5

In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance

# permuter 정의
permuter = PermutationImportance(
    xgb_pipe, #xgbregressor
    scoring='r2', # metric
    n_iter=5, # 다른 random seed를 사용하여 5번 반복
    random_state=2
)

permuter.fit(X_train, y_train)


feature_names = X_test.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None, 
    feature_names=feature_names 
)